### <center>Fixed Income Markets</center>
### <center>Homework 8</center>
#### <center>Group Members - Ashutosh Ekade, Zechen Liu, Jessica Ghai, Yuetong Li</center>

1. When it comes to how the swaps are valued in both cases, then it seems that the Lehman brothers are interpreting the swap agreements based on the conditions that were agreed upon before bankruptcy, emphasizing terms that would favor a lower termination payment or even a receivable in their favor. It is possible that Lehman Brothers is insisting on using the one-month Libor rate plus a specific margin as the benchmark for calculating the floating rate under the swap agreements, consistent with their understanding of the contractual terms before bankruptcy. On the other hand, for Giants Stadium LLC, we have that the company appears to be interpreting the swap agreements differently, potentially arguing that the termination caused by Lehman's bankruptcy allows for a different method of valuation, possibly invoking clauses that adjust the valuation in the event of default or bankruptcy. They might argue for a valuation that considers the market's response to the bankruptcy or different benchmarks for determining the interest rates post-termination. Lehman's valuation might assume more stable or pre-crisis market conditions, focusing on contractual protections they believe were in place to shield them from significant liabilities under the swaps. The company might be assuming lower market volatility or a narrower credit spread, affecting the fixed-to-floating rate differential. Conversely, Giants Stadium's valuation may reflect post-crisis market conditions, including higher volatility and wider credit spreads, especially those attributable to Lehman's bankruptcy. They might assume that the market's perception of risk has significantly changed, impacting the value of the swaps.

2. When it comes to the Lehman Brothers, one critique could be that Lehman's valuation overly relies on conditions or contractual terms that are favorable to them, which means that they are ignoring market realities or the specific impact their bankruptcy has had on the market. Their technique might not fully account for the actual market conditions at the time of termination, particularly the credit risk and market perception changes post-bankruptcy. On the other hand, the critique against Giants Stadium’s valuation could center on the possibility of inflating the swap's value by leveraging the bankruptcy scenario or interpreting the swap termination clauses in a way that significantly benefits them financially. They may be accused of selecting assumptions or market scenarios that unrealistically escalate the swap's termination value, perhaps by not properly accounting for the existing market rates or the true impact of Lehman's bankruptcy on the swap's conditions.

3. We have that traditional floating-rate debt typically has interest rates that are reset at regular intervals based on a reference rate like LIBOR. The primary risk for holders is the change in interest rates. If rates rise, the cost of borrowing for the issuer increases, but the value of the floating rate notes to the investor increases due to higher interest payments. On the other hand, auction rate floating rate debt (ARS) has its interest rates reset through periodic auctions. ARS can face liquidity issues if there's a failure in auction mechanisms, leading to interest rates that do not accurately reflect market conditions. Beyond the general interest rate risk, ARS holders face significant liquidity risk and auction failure risk. In the event of failed auctions, investors cannot liquidate their positions and could be stuck with a lower-than-expected interest rate. Moreover, during financial crises or times of high market stress, the risk of auction failures increases, which can lead to widespread valuation and liquidity issues for the securities.

4. Solution below

In [14]:
# Given data
swap_rates_15th = {
    '1M': 0.0250,
    '3M': 0.0282,
    '6M': 0.0300,
    '1Y': 0.0262,
    '2Y': 0.0293,
    '3Y': 0.0329,
    '4Y': 0.0354,
    '5Y': 0.0369,
    '7Y': 0.0398,
    '10Y': 0.0424,
    '30Y': 0.0461
}

swap_rates_18th = {
    '1M': 0.0319,
    '3M': 0.0320,
    '6M': 0.0339,
    '1Y': 0.0299,
    '2Y': 0.0295,
    '3Y': 0.0325,
    '4Y': 0.0346,
    '5Y': 0.0361,
    '7Y': 0.0388,
    '10Y': 0.0413,
    '30Y': 0.0447
}

notional_amount = 408325000
fixed_rate = 0.061885
maturity_year = 2047
valuation_year = 2008

# all hold is 90% of libor
all_hold_rate_15_sep = 0.90 * swap_rates_15th['1M']
all_hold_rate_18_sep = 0.90 * swap_rates_18th['1M']

# we will assume the 3-month LIBOR rate as the discount rate for all future cash flows
discount_rate_15_sep = swap_rates_15th['3M']
discount_rate_18_sep = swap_rates_18th['3M']

# find PV
def present_value(rate, days, notional):
    return (1 - rate * days / 360) * notional

# find NPV
def calculate_npv(rate, cash_flows):
    npv = 0
    for i, cash_flow in enumerate(cash_flows, start=1):
        npv += cash_flow / ((1 + rate) ** i)
    return npv

# years to maturity
ytm = maturity_year - valuation_year

# pv of fixed leg
fixed_leg_cash_flows = [fixed_rate * notional_amount] * ytm
fixed_leg_pv_15_sep = calculate_npv(discount_rate_15_sep, fixed_leg_cash_flows)
fixed_leg_pv_18_sep = calculate_npv(discount_rate_18_sep, fixed_leg_cash_flows)

# pv of floating leg
floating_leg_pv_15_sep = present_value(all_hold_rate_15_sep, 30, notional_amount)
floating_leg_pv_18_sep = present_value(all_hold_rate_18_sep, 30, notional_amount)

# value of swaps
swap_value_15_sep = fixed_leg_pv_15_sep - floating_leg_pv_15_sep
swap_value_18_sep = fixed_leg_pv_18_sep - floating_leg_pv_18_sep

print("Present value of the fixed leg as of September 15, 2008:", fixed_leg_pv_15_sep)
print("Present value of the fixed leg as of September 18, 2008:", fixed_leg_pv_18_sep)
print("Present value of the floating leg as of September 15, 2008:", floating_leg_pv_15_sep)
print("Present value of the floating leg as of September 18, 2008:", floating_leg_pv_18_sep)
print("Value of the swaps as of September 15, 2008:", swap_value_15_sep)
print("Value of the swaps as of September 18, 2008:", swap_value_18_sep)

Present value of the fixed leg as of September 15, 2008: 593159080.6687349
Present value of the fixed leg as of September 18, 2008: 558491353.8136289
Present value of the floating leg as of September 15, 2008: 407559390.625
Present value of the floating leg as of September 18, 2008: 407348082.4375
Value of the swaps as of September 15, 2008: 185599690.0437349
Value of the swaps as of September 18, 2008: 151143271.3761289


My assumptions: Assumed constant discount rate for all maturities for easier calculation.
I would have considered the given numbers for the respective tenures and interpolated the nearest discount factors for anything in between for which numbers are not available.

5. Solution below

In [13]:
# 1 month libor + spread
libor_rate_on_termination = swap_rates_18th['1M'] + 0.003185

# Calculate the floating leg cash flows with the new LIBOR rate plus the spread
# Assuming the first payment includes the additional spread and subsequent payments use the 3-month LIBOR rate.
floating_leg_cash_flows = [libor_rate_on_termination * notional_amount] + \
                          [swap_rates_18th['1M'] * notional_amount for year in range(ytm - 1)]

# Present value of the floating leg cash flows on the termination date
pv_floating_leg_on_termination = calculate_npv(discount_rate_18_sep, floating_leg_cash_flows)

# The value of the swaps is the present value of the fixed leg minus the present value of the floating leg on the termination date
swap_value_on_termination = fixed_leg_pv_18_sep - pv_floating_leg_on_termination

# Print the results
print("Present value of the fixed leg as of September 18, 2008:", fixed_leg_pv_18_sep)
print("Present value of the floating leg as of September 18, 2008:", pv_floating_leg_on_termination)
print("Value of the swaps on September 18, 2008:", swap_value_on_termination)

Present value of the fixed leg as of September 18, 2008: 558491353.8136289
Present value of the floating leg as of September 18, 2008: 289146982.1045027
Value of the swaps on September 18, 2008: 269344371.70912623


6. Solution below

In [11]:
# Given data from the user for the specific scenario of Giants Stadium
last_auction_rate = 10.17 / 100  # Convert percentage to decimal

# Calculate the cash flows for the floating leg based on the fixed floating rate
# Assuming the floating rate remains constant for the remaining life of the swap
floating_leg_cash_flows_fixed_rate = [last_auction_rate * notional_amount for _ in range(ytm)]

# Present value of the floating leg cash flows on the termination date using the fixed floating rate
pv_floating_leg_fixed_rate = calculate_npv(discount_rate_18_sep, floating_leg_cash_flows_fixed_rate)

# Calculate the value of the swaps on the termination date using Giants Stadium's method
swap_value_giants_method = fixed_leg_pv_18_sep - pv_floating_leg_fixed_rate

# Print the results
print("Present value of the fixed leg as of September 18, 2008:", fixed_leg_pv_18_sep)
print("Present value of the floating leg as of September 18, 2008 using the fixed floating rate:", pv_floating_leg_fixed_rate)
print("Value of the swaps as of September 18, 2008 using Giants Stadium's method:", swap_value_giants_method)

Present value of the fixed leg as of September 18, 2008: 558491353.8136289
Present value of the floating leg as of September 18, 2008 using the fixed floating rate: 917808365.2394934
Value of the swaps as of September 18, 2008 using Giants Stadium's method: -359317011.42586446


7. Factors that could influence the valuation of the swaps include the overall liquidity in the market, changes in credit spreads, particularly those of Lehman Brothers and related entities, the general availability of credit post-crisis, changes in the underlying rate (from auction rate to Libor, for instance), and legal uncertainties or contractual interpretations arising from Lehman's bankruptcy filing. The terms of collateralization and the specifics of swap agreements, including fallback provisions and benchmark rates, also play critical roles in determining the value of swaps in this new, more risk-averse environment. The unprecedented market conditions following the Lehman collapse, including significant shifts in interest rates and credit risk perceptions, would also play a critical role in re-evaluating the swaps' worth.